# Old or not

## Business Understanding: 
Van volwassen beroemdheden is het vaak lastig te zeggen hoe oud ze zijn. Nu gaan we kijken of we een neuraal netwerk kunnen bouwen wat dit beter kan schatten dan wij. We nemens als grens de compleet arbitraire waarde van 42 jaar. Iedereen tussen de 21 en 42 noemen we 'not old' en iedereen boven deze grens definieren we als 'old' 

We gebruiken hiervoor de IMDB faces database met daarin meer dan 460.000 gezichten van een dikke 20.000 beroemdheden afkomstig van IMDB (zie https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/ voor uitleg en details) en hierbij de IMDB informatie over o.a. leeftijd en geslacht. We downloaden hier de de cropped IMDB dataset( https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar). De metadata wordt in .mat (matlab) bestand meegeleverd. 

- We beperken ons tot volwassen beroemdheden. <u>Iedereen onder de 21 jaar mag je overslaan.</u>
- Er zitten nogal wat foto's met meerdere gezichten of geen gezicht op. <u>Deze mag je eruit filteren</u>
- We verwachten niet alleen de code maar (in het notebook) ook een <u>uitleg</u> van waarom je dingen hebt gedaan. 
- I.v.m. performance issues gebruiken we na opschoning alleen de <u>1e 50.000</u> records als data
- Gebruik <u>20% van je data als testset, gebruik 42 als random state</u>
- Eindig met de mogelijkheid om een jpg van bijv. jezelf in te lezen en d.m.v. het getrainde netwerk te bepalen of diegene wel of niet oud is.

Vul je antwoorden steeds op de juiste plaats in. Maak indien nodig extra cellen aan.

## Data Understanding: 

In [22]:
#importeer de juiste packages
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from scipy.io import loadmat

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import sys

sys.OpenCV_LOADER_DEBUG = True
import cv2

from PIL.JpegImagePlugin import JpegImageFile

In [2]:
#Lees de metadata in

mat = loadmat("imdb_crop/imdb.mat", squeeze_me=True)

In [3]:
# bestudeer deze en maak voor iedere relevante variabele een aparte list aan
mat: dict = mat
imdb: np.ndarray = mat["imdb"]

(dob, photo_taken, full_path, gender, name, face_location, face_score, second_face_score, celeb_names, celeb_id) = imdb.tolist()

dob1 = dob[0]
print(dob1, type(dob1))

def convertDate(date):
    EPOCH = 719529
    return pd.to_datetime(int(date) - EPOCH, unit='D')

693726 <class 'numpy.int32'>


## Data preparation: 

In [5]:
#Bepaal op basis van de metadata hoe oud de personen op de foto's waren toen deze genomen werden (-1 als onbekend)

age_on_photo = []
# invalids = 0
for i in range(0, len(dob)):
    try:
        date = convertDate(dob[i])
        ageThen = photo_taken[i] - date.year
        age_on_photo.append(ageThen)
    except Exception as e:
        # invalids += 1
        age_on_photo.append(-1)

# print(invalids, "zijn/is ongeldig, dit is", (float(invalids)/len(dob)*100), "procent")
age_on_photo

[69,
 71,
 69,
 69,
 69,
 67,
 80,
 80,
 67,
 80,
 84,
 67,
 50,
 83,
 50,
 82,
 82,
 82,
 82,
 80,
 64,
 80,
 50,
 83,
 50,
 81,
 83,
 83,
 81,
 84,
 83,
 67,
 82,
 82,
 86,
 84,
 66,
 80,
 80,
 67,
 50,
 67,
 80,
 82,
 81,
 67,
 32,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 28,
 28,
 28,
 28,
 29,
 29,
 29,
 29,
 29,
 32,
 32,
 32,
 32,
 29,
 32,
 85,
 85,
 85,
 85,
 54,
 54,
 54,
 54,
 59,
 59,
 54,
 54,
 63,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 77,
 72,
 66,
 86,
 66,
 66,
 66,
 70,
 66,
 66,
 42,
 55,
 43,
 53,
 87,
 40,
 66,
 72,
 72,
 52,
 52,
 52,
 77,
 52,
 70,
 48,
 48,
 48,
 48,
 70,
 70,
 66,
 55,
 66,
 54,
 72,
 38,
 38,
 47,
 43,
 53,
 38,
 41,
 41,
 41,
 38,
 38,
 38,
 38,
 44,
 38,
 44,
 44,
 44,
 38,
 44,
 44,
 44,
 38,
 44,
 38,
 44,
 43,
 38,
 38,
 59,
 59,
 39,
 44,
 44,
 44,
 44,
 38,
 59,
 52,
 39,
 52,
 39,
 59,
 39,
 39,
 52,
 59,
 52,


In [6]:
#voeg de ingelezen metadata en de berekende leeftijd samen tot een nieuwe dataset

df = pd.DataFrame(data={
    'name': name,
    'full_path': full_path,
    'age_on_photo': age_on_photo,
    'face_score': face_score,
    'second_face_score': second_face_score,
})

In [7]:
#schoon de dataset op op basis van de bij de business understanding gegeven criteria.

## Filter: check of mensen minsten 21 jaar oud zijn
df_filtered = df[df['age_on_photo'] >= 21]

## Filter: alleen foto's waar gezichten op gevonden zijn
#       face_score: detector score (the higher the better).
#                   Inf implies that no face was found in the image and the face_location then just returns the entire image
df_filtered = df_filtered[df_filtered['face_score'] != np.Infinity]

## Filter: alleen foto's waar maar een gezicht op staat
##      second_face_score: detector score of the face with the second highest score.
#                          This is useful to ignore images with more than one face.
#                          second_face_score is NaN if no second face was detected.
df_filtered = df_filtered[pd.isna(df_filtered['second_face_score'])]

## Filter: max 50000
df_filtered = df_filtered.head(50000)

df_filtered.info()
df_filtered.head()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 3 to 98166
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               50000 non-null  object 
 1   full_path          50000 non-null  object 
 2   age_on_photo       50000 non-null  int64  
 3   face_score         50000 non-null  float64
 4   second_face_score  0 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 2.3+ MB


,name,full_path,age_on_photo,face_score,second_face_score
3,Fred Astaire,01/nm0000001_rm946909184_1899-5-10_1968.jpg,69,1.872117,NaN
4,Fred Astaire,01/nm0000001_rm980463616_1899-5-10_1968.jpg,69,1.158766,NaN
6,Lauren Bacall,02/nm0000002_rm1346607872_1924-9-16_2004.jpg,80,3.479189,NaN
7,Lauren Bacall,02/nm0000002_rm1363385088_1924-9-16_2004.jpg,80,3.870171,NaN
12,Lauren Bacall,02/nm0000002_rm221957120_1924-9-16_1974.jpg,50,4.096431,NaN


In [8]:
# Lees de afbeeldingen in

images = [cv2.imread("imdb_crop/" + path) for path in df_filtered['full_path']]

In [11]:
widths = np.array([img.shape[0] for img in images])
heights = np.array([img.shape[1] for img in images])

print(f"Width van {widths.min()} naar {widths.max()}")
print(f"Height van {heights.min()} naar {heights.max()}")


Width van 31 naar 501
Height van 31 naar 501


## Modeling: 

In [12]:
WIDTH = HEIGHT = 100
SIZE = (WIDTH, HEIGHT)
images_resized = np.array([cv2.resize(img, SIZE) for img in images])

In [13]:
images_resized_scaled = images_resized / 255.0


In [23]:
train_images, test_images = train_test_split(images_resized_scaled, test_size=0.2, random_state=42)

: 

In [18]:
images_resized_scaled.shape

(50000, 100, 100, 3)

In [ ]:
# train het model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(WIDTH, HEIGHT, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dropout(0.1),
    layers.Dense(10, activation='softmax'),
])


model.compile(optimizer="ADAM",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

train_datagen = ImageDataGenerator(
    # rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    # vertical_flip=True,
)

data_flow = train_datagen.flow(train_images, tf.reshape(train_labels, (-1)), shuffle=True)
history2 = model.fit(data_flow, epochs=25, validation_data=(test_images, test_labels))

In [ ]:
# en valideer dit op de testdata

## Evaluation: 

In [ ]:
# plot de accuracy en validated accuracy, toon de confusion matrix,
# en toon de een aantal afbeeldingen inclusief of ze volgens het model wel of niet oud zijn,
# doe dit laatste ook voor een paar eigen afbeeldingen


Kunnen we met dit model betrouwbaar schatten of een beroemdheid oud is?
(type je antwoord en uitleg hieronder)